# Linear Models
This notebook compares regression in C results to those from scipy and statsmodels for linear models. This includes both real-valued and complex-valued data.

First define some functions that will help do the comparisons.



In [1]:
from regressioninc.linear.models import OLS, WLS, MEstimator
from scipy.linalg import lstsq
import statsmodels.api as sm
import pandas as pd


def tabulate_comparison(rinc_params, compare_params):
    """Tabulate the comparison of estimated parameters"""
    diff = rinc_params - compare_params
    data = {}
    for ip in range(rinc_params.size):
        data[f"param {ip + 1}"] = [rinc_params[ip], compare_params[ip], diff[ip]]
    df = pd.DataFrame(data=data, index=["Regression in C", "Comparison", "Difference"])
    return df


def add_actual(df, real_params, intercept=None):
    """Add the actual parameters to the tabulation"""
    if intercept is not None:
        real_params = list(real_params) + [intercept]
    data = {f"param {ip + 1}": real_params[ip] for ip in range(len(real_params))}
    df_actual = pd.DataFrame(data=data, index=["Actual"])
    return pd.concat((df_actual, df))


def compare_ols(X, y):
    """Return parameters from regressioninc and scipy"""
    # regression in C
    rinc_model = OLS()
    rinc_model.fit(X, y)
    rinc_params = rinc_model.estimate.params
    # scipy
    scipy_params, _residues, _rank, _s = lstsq(X, y)
    return tabulate_comparison(rinc_params, scipy_params)


def compare_wls(X, y, weights):
    """Return parameters from regressionc and statsmodels"""
    # regression in C
    rinc_model = WLS()
    rinc_model.fit(X, y, weights=weights)
    rinc_params = rinc_model.estimate.params
    # statsmodels
    sm_wls = sm.WLS(y, X, weights=weights)
    wls_results = sm_wls.fit()
    return tabulate_comparison(rinc_params, wls_results.params)


def compare_m_estimate(X, y, M1):
    """Return parameters from regressioninc and statsmodels"""
    rinc_model = MEstimator(warm_start=True)
    rinc_model.fit(X, y, M=M1)
    rinc_estimate_1 = rinc_model.estimate.params
    # statsmodels
    sm_rlm = sm.RLM(y, X, M=M1)
    rlm_result = sm_rlm.fit(maxiter=50, tol=1e-8, scale_est="mad", conv="sresid")
    sm_rlm_1 = rlm_result.params
    return tabulate_comparison(rinc_estimate_1, sm_rlm_1)


def compare_mm_estimate(X, y, M1, M2):
    """Return parameters from regressioninc and statsmodels"""
    rinc_model = MEstimator(warm_start=True)
    rinc_model.fit(X, y, M=M1)
    rinc_estimate_1 = rinc_model.estimate.params
    rinc_model.fit(X, y, M=M2)
    rinc_estimate_2 = rinc_model.estimate.params
    # statsmodels
    sm_rlm = sm.RLM(y, X, M=M1)
    rlm_result = sm_rlm.fit(maxiter=50, tol=1e-8, scale_est="mad", conv="sresid")
    sm_rlm_1 = rlm_result.params
    rlm = sm.RLM(y, X, M=M2)
    rlm_result = rlm.fit(
        maxiter=50,
        tol=1e-8,
        scale_est="mad",
        conv="sresid",
        start_params=sm_rlm_1,
    )
    sm_rlm_2 = rlm_result.params
    return tabulate_comparison(rinc_estimate_2, sm_rlm_2)

## Real-valued linear models

In [2]:
import numpy as np
from regressioninc.testing.real import generate_linear, add_gaussian_noise
from regressioninc.linear.models import add_intercept

np.random.seed(42)
ADD_NOISE = True

# generate the example data
params = np.array([7.6, -3.8])
intercept = -6
X, y = generate_linear(params, 100, intercept=intercept)
X = add_intercept(X)
if ADD_NOISE:
    y = add_gaussian_noise(y, 0, 3)

print("OLS")
df = compare_ols(X, y)
df = add_actual(df, params, intercept=intercept)
print(df)

print("\n\nWLS")
weights = np.ones_like(y, dtype=float)
df = compare_wls(X, y , weights)
df = add_actual(df, params, intercept=intercept)
print(df)

print("\n\nM Estimator")
M1 = sm.robust.norms.TukeyBiweight()
df = compare_m_estimate(X, y, M1)
df = add_actual(df, params, intercept=intercept)
print(df)

print("\n\nMM Estimator")
M2 = sm.robust.norms.TrimmedMean()
df = compare_mm_estimate(X, y, M1, M2)
df = add_actual(df, params, intercept=intercept)
print(df)



2023-10-08 21:02:53.307 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking
2023-10-08 21:02:53.314 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking
2023-10-08 21:02:53.315 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking


OLS
                 param 1   param 2   param 3
Actual            7.6000 -3.800000 -6.000000
Regression in C   7.6254 -3.773379 -5.642764
Comparison        7.6254 -3.773379 -5.642764
Difference        0.0000  0.000000  0.000000


WLS
                      param 1       param 2       param 3
Actual           7.600000e+00 -3.800000e+00 -6.000000e+00
Regression in C  7.625400e+00 -3.773379e+00 -5.642764e+00
Comparison       7.625400e+00 -3.773379e+00 -5.642764e+00
Difference       8.881784e-16  1.776357e-15 -4.440892e-15


M Estimator
                      param 1       param 2       param 3
Actual           7.600000e+00 -3.800000e+00 -6.000000e+00
Regression in C  7.620207e+00 -3.789905e+00 -5.770261e+00
Comparison       7.620207e+00 -3.789905e+00 -5.770261e+00
Difference       3.376321e-11  1.362244e-10  9.304326e-10


MM Estimator
                      param 1       param 2       param 3
Actual           7.600000e+00 -3.800000e+00 -6.000000e+00
Regression in C  7.623338e+00 -3.810616e

## Complex-valued linear models

Let's now compare regression in C linear models to other packages for complex-valued data. Scipy lstsq does support complex-valued data so no differences are expected there. However, for other models, results might not be the same.

In [3]:
from regressioninc.testing.complex import ComplexGrid, generate_linear_grid
from regressioninc.testing.complex import add_gaussian_noise
from regressioninc.linear.models import add_intercept

np.random.seed(42)
ADD_NOISE = True

params = np.array([0.5 + 2j, -3 - 1j])
intercept = 20 + 20j
grid_r1 = ComplexGrid(r1=0, r2=10, nr=11, i1=-5, i2=5, ni=11)
grid_r2 = ComplexGrid(r1=-25, r2=-5, nr=11, i1=-5, i2=5, ni=11)
X, y = generate_linear_grid(params, [grid_r1, grid_r2], intercept=intercept)
X = add_intercept(X)
if ADD_NOISE:
    y = add_gaussian_noise(y, loc=(0, 0), scale=(3, 3))

print("OLS")
df = compare_ols(X, y)
df = add_actual(df, params, intercept)
print(df)

print("\n\nWLS")
weights = np.ones_like(y, dtype=float)
df = compare_wls(X, y , weights)
df = add_actual(df, params, intercept)
print(df)

print("\n\nM estimates")
M1 = sm.robust.norms.TukeyBiweight()
df = compare_m_estimate(X, y, M1)
df = add_actual(df, params, intercept)
print(df)

print("\n\nM estimates")
M1 = sm.robust.norms.TrimmedMean()
df = compare_m_estimate(X, y, M1)
df = add_actual(df, params, intercept)
print(df)

print("\n\nMM estimates")
M1 = sm.robust.norms.TukeyBiweight()
M2 = sm.robust.norms.TrimmedMean()
df = compare_mm_estimate(X, y, M1, M2)
df = add_actual(df, params, intercept)
print(df)


2023-10-08 21:03:01.315 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking


/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/numpy/core/_asarray.py:130: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
2023-10-08 21:03:01.324 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking
/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/numpy/core/_asarray.py:130: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
2023-10-08 21:03:01.331 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking
2023-10-08 21:03:01.332 | DEBUG    | regressioninc.linear.models:fit:364 - Early stopping criteria met, breaking


OLS
                            param 1             param 2               param 3
Actual           0.500000+2.000000j -3.000000-1.000000j  20.000000+20.000000j
Regression in C  0.416040+1.964015j -2.949279-0.955329j  21.072774+20.942814j
Comparison       0.416040+1.964015j -2.949279-0.955329j  21.072774+20.942814j
Difference       0.000000+0.000000j  0.000000+0.000000j  0.0000000+0.0000000j


WLS
                                    param 1                     param 2  \
Actual           5.000000e-01+2.000000e+00j -3.000000e+00-1.000000e+00j   
Regression in C  4.160404e-01+1.964015e+00j -2.949279e+00-9.553286e-01j   
Comparison       4.160404e-01+1.964015e+00j -2.949279e+00-9.553286e-01j   
Difference      -2.553513e-15+8.881784e-16j  1.021405e-14-5.551115e-16j   

                                    param 3  
Actual           2.000000e+01+2.000000e+01j  
Regression in C  2.107277e+01+2.094281e+01j  
Comparison       2.107277e+01+2.094281e+01j  
Difference       7.105427e-15-1.421085e-

/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/numpy/core/_asarray.py:130: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:255: ComplexWarning: Casting complex values to real discards the imaginary part
  start_params = np.asarray(start_params, dtype=np.double).squeeze()
/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:174: RuntimeWarning: divide by zero encountered in divide
  history['sresid'].append(tmp_results.resid / tmp_results.scale)
